# Решение тестового задания на DS стажировку в Avito

Задача: хотим угадывать, где пропущен пробел в данных.

Краткий обзор методов:

1) Языковые модели(RoBERTa, BERT, T5) -- использовать из коробки, заствляя BERT поедсказывать пробельный токен не получилось. Закономерный шаг -- затюнить. Ограничения с моей стороны -- долго обучать. Сбор данных вышел бы простой -- набрать тектов, повыбирать оттуда чанки предложений на 2-7 слов.

2) Использование токенизаторов -- тут не удалось докуртить идею до чего-то разумного, строка может разбиваться на токены самыми странными способами

3) ДП, частотные словари и эвристики показались довольно перспективным вариантом для, по крайней мере, бейзлайна.



In [ ]:
import pandas as pd
from io import StringIO
import math

In [ ]:
# Загружаем частотный словарь (с трудом нашел в интернете по этой ссылке https://www.artint.ru/projects/frqlist/frqlist-en.php?utm_source=chatgpt.com)
# Краткое описание: содержит порядка 32К слов с статистикой встречаемости. Собран по большому корпусу текстов.
def load_freq_dict(path="./words.num"):
    freqs = {}
    total = 0
    with open(path, 'r', encoding='cp1251') as f:
        for line in f:
            num, freq, word = line.strip().split()
            freq = int(float(freq))
            freqs[word] = freq
            total += freq
    # нормализуем и логарифмируем, т.к. работаем с маленькими числами
    probs = {w: math.log(f / total) for w, f in freqs.items()}
    return probs

# Алгоритм сегментации через динамическое программирование
def segment(text, probs, max_word_len=20):
    n = len(text)
    dp = [-float("inf")] * (n + 1)   # хранит максимальный логарифм вероятности разбиения подстроки text[:i]
    back = [-1] * (n + 1)           # хранит индекс, откуда мы пришли в i(будем восстанавливать сегментацию)
    dp[0] = 0

    for i in range(1, n + 1): # индекс начала слова
        for l in range(1, min(max_word_len, i) + 1): # длина слова
            word = text[i - l:i]
            if word in probs:
                score = dp[i - l] + probs[word] # копим сумму логарифмов правдоподобия
                if score > dp[i]:
                    dp[i] = score
                    back[i] = i - l # запомнили, где началось слово

    # восстановаем разбиение
    words = []
    i = n
    while i > 0 and back[i] != -1:
        words.append(text[back[i]:i])
        i = back[i]
    words.reverse()
    return words

# проверим, что работает
probs = load_freq_dict()

text = "елкиипалки"
print(" ".join(segment(text, probs)))

Метод выше прямо использоваться не будем, мы обвесим его эвристиками.

Мотивация эвристик:

1) Есть слова на латинице, которые мешают обработки(плюс понятно, что это слова собственные и их надо выделять)
2) -, будем считать, выступает как в конструкции "объект - это объект"
3) после запятой пробел, до неё -- нет
4) цифры тоже выделяем отдельно

In [ ]:
import re

def predict_space_positions(text, probs):
    positions = []
    text = text.lower() # нужно привести в нижний регистр, чтобы всё работало

    # 1) Эвристики
    for i, ch in enumerate(text):
        # 1.1) цифры
        if ch.isdigit():
            if i > 0 and text[i-1].isalpha():
                positions.append(i)
            if i+1 < len(text) and text[i+1].isalpha():
                positions.append(i+1)

        # 1.2) латиница
        if ch.isalpha() and ('a' <= ch.lower() <= 'z'):
            if i > 0 and text[i-1].isalpha() and ('а' <= text[i-1].lower() <= 'я'):
                positions.append(i)
            if i+1 < len(text) and ('а' <= text[i+1].lower() <= 'я'):
                positions.append(i+1)

        # 1.3) запятая
        if ch == ',' and i+1 < len(text) and text[i+1] != ' ':
            positions.append(i+1)

        # 1.4) дефис
        if ch == '-':
            if i > 0 and text[i-1] != ' ':
                positions.append(i)
            if i+1 < len(text) and text[i+1] != ' ':
                positions.append(i+1)

    # Алгоритм с ДП сегментацией
    clean = re.sub(r'[^а-яА-Я]', '', text.lower()) # Удаляем все, о чем говорили выше, чтобы не мешало обработке
    words = segment(clean, probs)
    idx = 0
    for w in words[:-1]:
        idx = text.find(w, idx) + len(w)
        if idx < len(text) and text[idx] != ',': # перед запятой не может быть пробела
            positions.append(idx)

    # на всякий случай, убираем будли
    return sorted(set(positions))


test_predict = 'куплюxiaomi5500'
predict_space_positions(test_predict, probs)

In [ ]:
with open('./dataset_1937770_3.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

processed_lines = []
for line in lines:
    parts = line.strip().split(',', 1)
    if len(parts) == 2:
        processed_lines.append(parts)

task_data = pd.DataFrame(processed_lines[1:], columns=processed_lines[0])
task_data.head()

In [ ]:
task_data['predicted_positions'] = task_data['text_no_spaces'].apply(lambda x: str(predict_space_positions(x, probs)))
task_data.head(10)

In [ ]:
to_submit = task_data.drop('text_no_spaces', axis=1)
to_submit.to_csv('submit_dp_heur_1.txt')

# Итог

Получили бейзлайн, показывающий на паблике 87,5% при том с очень дешевым инференсом, работающим за линейное время.

Интернет говорит, что это похоже на алгоритм Витерби.